In [1]:
# file to compare number of queries per pcap file


# chrony-based VM with NTS

* This will emulate exaclty what Ubuntu will do with 25.10



## 1. VM Setup

```shell

$ multipass launch --name my-vm
$ multipass shell my-vm

$lsb_release -a
No LSB modules are available.
Distributor ID: Ubuntu
Description:    Ubuntu 24.04.3 LTS
Release:        24.04
Codename:       noble


$ sudo apt install chrony

```


## 2.  Configuring Chrony with NTS

* First, you need to install `chrony`
* Chrony on `Ubuntu 24.04.3 LTS` does not have NTS enabled by default
* Out goal is to emulate _exactly_ the new setup by Ubuntu with NTS
* To do what, we do the following:
   1. Go to Ubuntu's package site: https://packages.ubuntu.com/questing/chrony
   2. Download the package directly: [chrony_4.7-1ubuntu1.debian.tar.xz]
   3. Extract it and copy the important settings to `/etc/chrony`


### 2.1 `chrony.conf` 

```shell
$ cat /etc/chrony/chrony.conf
# Welcome to the chrony configuration file. See chrony.conf(5) for more
# information about usable directives.

# The Ubuntu NTP pool servers configuration was moved to /etc/chrony/sources.d/ubuntu-ntp-pools.sources

# Use time sources from DHCP.
# Those could be non-NTS sources. If you want to avoid unauthenticated NTP
# sources comment the following line. (LP: #2115565)
sourcedir /run/chrony-dhcp

# Use NTP sources found in /etc/chrony/sources.d.
sourcedir /etc/chrony/sources.d

# This directive specifies the location of the file containing ID/key pairs for
# NTP authentication.
keyfile /etc/chrony/chrony.keys

# This directive specifies the file into which chronyd will store the rate
# information.
driftfile /var/lib/chrony/chrony.drift

# Save NTS keys and cookies.
ntsdumpdir /var/lib/chrony

# Uncomment the following line to turn logging on.
#log tracking measurements statistics

# Log files location.
logdir /var/log/chrony

# Stop bad estimates upsetting machine clock.
maxupdateskew 100.0

# This directive enables kernel synchronisation (every 11 minutes) of the
# real-time clock. Note that it can't be used along with the 'rtcfile' directive.
rtcsync

# Step the system clock instead of slewing it if the adjustment is larger than
# one second, but only in the first three clock updates.
makestep 1 3

# Get TAI-UTC offset and leap seconds from the system tz database.
# This directive must be commented out when using time sources serving
# leap-smeared time.
#leapseclist /usr/share/zoneinfo/leap-seconds.list

# Include configuration files found in /etc/chrony/conf.d.
confdir /etc/chrony/conf.d

````


## 2.2 `sources.d/ubuntu-ntp-pools.sources `

```shell
$ cat  sources.d/ubuntu-ntp-pools.sources
# Use NTS by default
# NTS uses an additional port to negotiate security: 4460/tcp
# The normal NTP port remains in use: 123/udp
pool 1.ntp.ubuntu.com iburst maxsources 1 nts prefer
pool 2.ntp.ubuntu.com iburst maxsources 1 nts prefer
pool 3.ntp.ubuntu.com iburst maxsources 1 nts prefer
pool 4.ntp.ubuntu.com iburst maxsources 1 nts prefer
# The bootstrap server is needed by systems without a hardware clock, or a very
# large initial clock offset. The specified certificate set is defined in
# /etc/chrony/conf.d/ubuntu-nts.conf.
pool ntp-bootstrap.ubuntu.com iburst maxsources 1 nts certset 1
```

## 2.3 Configuring NTS

* You need to copy multiple files to make NTS works:
   * `nts-bootstrap-staging-ubuntu.crt`
   * `nts-bootstrap-ubuntu.crt`
   * `conf.d/ubuntu-nts.conf` 




 

## 2.4 chrony status

```shell
$ chronyc sources

MS Name/IP address         Stratum Poll Reach LastRx Last sample               
===============================================================================
===============================================================================
^+ ntp-nts-2.ps5.canonical.>     2  10   361   22m   -112us[  -98us] +/-   10ms
^* ntp-nts-3.ps5.canonical.>     2  10   257   21m   +292us[ +307us] +/- 7279us
^- ntp-nts-2.ps6.canonical.>     2  10   377  1043  +4011us[+4011us] +/-   51ms
^- ntp-nts-3.ps6.canonical.>     2   9   377   415    -49us[  -49us] +/-   50ms
^- ntp-nts-1.ps6.canonical.>     2   9   377   159  +6474us[+6474us] +/-   55ms


$ chronyc -n sources 
MS Name/IP address         Stratum Poll Reach LastRx Last sample               
===============================================================================
^+ 185.125.190.122               2  10   361   22m   -112us[  -98us] +/-   10ms
^* 185.125.190.123               2  10   257   22m   +292us[ +307us] +/- 7279us
^- 91.189.91.112                 2  10   377  1071  +4011us[+4011us] +/-   51ms
^- 91.189.91.113                 2   9   377   443    -49us[  -49us] +/-   50ms
^- 91.189.91.111                 2   9   377   187  +6474us[+6474us] +/-   55ms

^ = server
* = currently selected source for synchronization
+ = acceptable source
- = source not used
? = unreachable or not yet reachable
x = source with a false tick
~ = source with a variable rate

```

# timesyncd setup


## 1. VM Setup

```shell

$ multipass launch --name timesyncd
$ multipass shell my-vm

$ lsb_release -a
No LSB modules are available.
Distributor ID: Ubuntu
Description:    Ubuntu 24.04.3 LTS
Release:        24.04
Codename:       noble
```





## 2.  inspecting timesyncd configuration

``` shell 

cat  /etc/systemd/timesyncd.conf 
#  This file is part of systemd.
#
#  systemd is free software; you can redistribute it and/or modify it under the
#  terms of the GNU Lesser General Public License as published by the Free
#  Software Foundation; either version 2.1 of the License, or (at your option)
#  any later version.
#
# Entries in this file show the compile time defaults. Local configuration
# should be created by either modifying this file (or a copy of it placed in
# /etc/ if the original file is shipped in /usr/), or by creating "drop-ins" in
# the /etc/systemd/timesyncd.conf.d/ directory. The latter is generally
# recommended. Defaults can be restored by simply deleting the main
# configuration file and all drop-ins located in /etc/.
#
# Use 'systemd-analyze cat-config systemd/timesyncd.conf' to display the full config.
#
# See timesyncd.conf(5) for details.

[Time]
#NTP=
#FallbackNTP=ntp.ubuntu.com
#RootDistanceMaxSec=5
#PollIntervalMinSec=32
#PollIntervalMaxSec=2048
#ConnectionRetrySec=30
#SaveIntervalSec=60
```

## timesyncd statuts

```shell

               Local time: Tue 2025-09-09 13:40:57 CEST
           Universal time: Tue 2025-09-09 11:40:57 UTC
                 RTC time: Tue 2025-09-09 11:40:57
                Time zone: Europe/Amsterdam (CEST, +0200)
System clock synchronized: yes
              NTP service: active
          RTC in local TZ: no

```

# Source code to compare pcaps

In [2]:

# function to parse pcap files captured from VMs

from scapy.all import rdpcap, UDP
from collections import defaultdict


# Reads a PCAP file and analyzes NTP traffic, summarizing per client-server pair:
# number of queries/responses, total sizes, and time between first and last query.
def analyze_ntp_pcap(pcap_file):
    packets = rdpcap(pcap_file)
    ntp_data = defaultdict(lambda: {
        'number_of_queries': 0,
        'number_of_responses': 0,
        'size_of_queries': 0,
        'size_of_responses': 0,
        'first_query_time': None,
        'last_query_time': None
    })

    for pkt in packets:
        if pkt.haslayer(UDP) and (pkt[UDP].sport == 123 or pkt[UDP].dport == 123):
            src_ip = pkt[0][1].src
            dst_ip = pkt[0][1].dst
            length = len(pkt)
            timestamp = pkt.time

            if pkt[UDP].dport == 123:  # NTP query
                key = (src_ip, dst_ip)
                ntp_data[key]['number_of_queries'] += 1
                ntp_data[key]['size_of_queries'] += length
                if ntp_data[key]['first_query_time'] is None:
                    ntp_data[key]['first_query_time'] = timestamp
                ntp_data[key]['last_query_time'] = timestamp
            elif pkt[UDP].sport == 123:  # NTP response
                key = (dst_ip, src_ip)
                ntp_data[key]['number_of_responses'] += 1
                ntp_data[key]['size_of_responses'] += length

    # Final formatting and sorting
    result = []
    for (client, server), data in ntp_data.items():
        time_diff = None
        queries_per_hour = None
        responses_per_hour = None

        if data['first_query_time'] and data['last_query_time']:
            time_diff = data['last_query_time'] - data['first_query_time']
            hours = time_diff / 3600 if time_diff > 0 else 1  # avoid division by zero
            queries_per_hour = data['number_of_queries'] / hours
            responses_per_hour = data['number_of_responses'] / hours

        result.append({
            'client': client,
            'ntp_server_ip': server,
            'number_of_queries': data['number_of_queries'],
            'number_of_responses': data['number_of_responses'],
            'size_of_queries': data['size_of_queries'],
            'size_of_responses': data['size_of_responses'],
            'time_between_first_and_last_query': time_diff,
            'queries_per_hour': queries_per_hour,
            'responses_per_hour': responses_per_hour
        })

    # Sort by number_of_queries descending
    result.sort(key=lambda x: x['number_of_queries'], reverse=True)

    return result



# Prints the NTP summary data as a Markdown-formatted table, sorted by number of queries in descending order.
# Adds a "Total" row that sums all queries, responses, sizes, and computes average query/response rates per hour.
def print_ntp_summary_markdown(summary_data):
    # Sort by number_of_queries descending
    sorted_data = sorted(summary_data, key=lambda x: x['number_of_queries'], reverse=True)

    # Add new columns headers
    print("| Client IP | NTP Server IP | # Queries | # Responses | Query Size (bytes) | Response Size (bytes) | Avg Query Size | Avg Response Size | Time Span (s) | Queries/hr | Responses/hr |")
    print("|-----------|----------------|-----------|--------------|--------------------|--------------------|----------------|-----------------|---------------|------------|--------------|")

    total_queries = 0
    total_responses = 0
    total_query_size = 0
    total_response_size = 0
    total_time_span = 0
    valid_time_spans = 0  # count of entries with valid time spans

    for entry in sorted_data:
        time_span = entry['time_between_first_and_last_query']
        time_span_hours = time_span / 3600 if time_span else None
        queries_per_hour = entry['number_of_queries'] / time_span_hours if time_span_hours else 0
        responses_per_hour = entry['number_of_responses'] / time_span_hours if time_span_hours else 0

        # Compute average sizes safely
        avg_query_size = entry['size_of_queries'] / entry['number_of_queries'] if entry['number_of_queries'] > 0 else 0
        avg_response_size = entry['size_of_responses'] / entry['number_of_responses'] if entry['number_of_responses'] > 0 else 0

        time_diff_str = f"{time_span:.2f}" if time_span else "N/A"
        qph_str = f"{queries_per_hour:.2f}" if time_span_hours else "N/A"
        rph_str = f"{responses_per_hour:.2f}" if time_span_hours else "N/A"
        avg_query_str = f"{avg_query_size:.2f}"
        avg_resp_str = f"{avg_response_size:.2f}"

        print(f"| {entry['client']} | {entry['ntp_server_ip']} | {entry['number_of_queries']} | {entry['number_of_responses']} | {entry['size_of_queries']} | {entry['size_of_responses']} | {avg_query_str} | {avg_resp_str} | {time_diff_str} | {qph_str} | {rph_str} |")

        # Accumulate totals
        total_queries += entry['number_of_queries']
        total_responses += entry['number_of_responses']
        total_query_size += entry['size_of_queries']
        total_response_size += entry['size_of_responses']
        if time_span:
            total_time_span += time_span
            valid_time_spans += 1

    # Compute average time span and rates
    avg_time_span_hours = (total_time_span / valid_time_spans) / 3600 if valid_time_spans else None
    total_qph = total_queries / avg_time_span_hours if avg_time_span_hours else 0
    total_rph = total_responses / avg_time_span_hours if avg_time_span_hours else 0
    avg_time_span_str = f"{total_time_span:.2f}" if valid_time_spans else "N/A"
    total_qph_str = f"{total_qph:.2f}" if avg_time_span_hours else "N/A"
    total_rph_str = f"{total_rph:.2f}" if avg_time_span_hours else "N/A"

    # Compute totals for average sizes
    total_avg_query_size = total_query_size / total_queries if total_queries > 0 else 0
    total_avg_response_size = total_response_size / total_responses if total_responses > 0 else 0

    total_avg_query_str = f"{total_avg_query_size:.2f}"
    total_avg_response_str = f"{total_avg_response_size:.2f}"

    # Print total line
    print(f"| **Total** | - | {total_queries} | {total_responses} | {total_query_size} | {total_response_size} | {total_avg_query_str} | {total_avg_response_str} | {avg_time_span_str} | {total_qph_str} | {total_rph_str} |")
    
    
    

from scapy.all import rdpcap, TCP
from collections import defaultdict

def analyze_nts_tcp_pcap(pcap_file):
    """
    Analyze TCP port 4460 packets (NTS), computing:
    - Number of queries/responses
    - Total sizes
    - Time between first and last query
    """
    packets = rdpcap(pcap_file)
    
    nts_data = defaultdict(lambda: {
        'number_of_queries': 0,
        'number_of_responses': 0,
        'size_of_queries': 0,
        'size_of_responses': 0,
        'first_query_time': None,
        'last_query_time': None
    })

    for pkt in packets:
        if pkt.haslayer(TCP):
            sport = pkt[TCP].sport
            dport = pkt[TCP].dport
            #print('got here')

            # Only TCP port 4460 packets
            if sport != 4460 and dport != 4460:
                continue

            src_ip = pkt[0][1].src
            dst_ip = pkt[0][1].dst
            length = len(pkt)
            timestamp = pkt.time

            if dport == 4460:  # query from client to server
                key = (src_ip, dst_ip)
                nts_data[key]['number_of_queries'] += 1
                nts_data[key]['size_of_queries'] += length
                if nts_data[key]['first_query_time'] is None:
                    nts_data[key]['first_query_time'] = timestamp
                nts_data[key]['last_query_time'] = timestamp
            elif sport == 4460:  # response from server to client
                key = (dst_ip, src_ip)
                nts_data[key]['number_of_responses'] += 1
                nts_data[key]['size_of_responses'] += length

    # Format results
    result = []
    for (client, server), data in nts_data.items():
        time_diff = None
        queries_per_hour = None
        responses_per_hour = None

        if data['first_query_time'] and data['last_query_time']:
            time_diff = data['last_query_time'] - data['first_query_time']
            hours = time_diff / 3600 if time_diff > 0 else 1
            queries_per_hour = data['number_of_queries'] / hours
            responses_per_hour = data['number_of_responses'] / hours

        result.append({
            'client': client,
            'ntp_server_ip': server,
            'number_of_queries': data['number_of_queries'],
            'number_of_responses': data['number_of_responses'],
            'size_of_queries': data['size_of_queries'],
            'size_of_responses': data['size_of_responses'],
            'time_between_first_and_last_query': time_diff,
            'queries_per_hour': queries_per_hour,
            'responses_per_hour': responses_per_hour
        })

    # Sort by number_of_queries descending
    result.sort(key=lambda x: x['number_of_queries'], reverse=True)
    return result


# Results

* I captured traffic on both VMs
* We compare them

In [3]:
#first, timesyncd

data=analyze_ntp_pcap('timesyncd.pcap')
print_ntp_summary_markdown(data)

| Client IP | NTP Server IP | # Queries | # Responses | Query Size (bytes) | Response Size (bytes) | Avg Query Size | Avg Response Size | Time Span (s) | Queries/hr | Responses/hr |
|-----------|----------------|-----------|--------------|--------------------|--------------------|----------------|-----------------|---------------|------------|--------------|
| 10.35.191.133 | 185.125.190.56 | 13 | 13 | 1170 | 1170 | 90.00 | 90.00 | 4995.25 | 9.37 | 9.37 |
| **Total** | - | 13 | 13 | 1170 | 1170 | 90.00 | 90.00 | 4995.25 | 9.37 | 9.37 |


* I just copied the previous output here to render as markdown

| Client IP | NTP Server IP | # Queries | # Responses | Query Size (bytes) | Response Size (bytes) | Avg Query Size | Avg Response Size | Time Span (s) | Queries/hr | Responses/hr |
|-----------|----------------|-----------|--------------|--------------------|--------------------|----------------|-----------------|---------------|------------|--------------|
| 10.35.191.133 | 185.125.190.56 | 13 | 13 | 1170 | 1170 | 90.00 | 90.00 | 4995.25 | 9.37 | 9.37 |
| **Total** | - | 13 | 13 | 1170 | 1170 | 90.00 | 90.00 | 4995.25 | 9.37 | 9.37 |

In [4]:
# second, chrony

data2=analyze_ntp_pcap('final-nts-ntp.pcap')
print_ntp_summary_markdown(data2)

| Client IP | NTP Server IP | # Queries | # Responses | Query Size (bytes) | Response Size (bytes) | Avg Query Size | Avg Response Size | Time Span (s) | Queries/hr | Responses/hr |
|-----------|----------------|-----------|--------------|--------------------|--------------------|----------------|-----------------|---------------|------------|--------------|
| 10.35.191.89 | 91.189.91.113 | 45 | 43 | 10486 | 10026 | 233.02 | 233.16 | 7623.27 | 21.25 | 20.31 |
| 10.35.191.89 | 91.189.91.112 | 40 | 40 | 9200 | 9200 | 230.00 | 230.00 | 7523.33 | 19.14 | 19.14 |
| 10.35.191.89 | 91.189.91.111 | 40 | 40 | 9200 | 9200 | 230.00 | 230.00 | 8407.89 | 17.13 | 17.13 |
| 10.35.191.89 | 185.125.190.123 | 39 | 33 | 9446 | 7998 | 242.21 | 242.36 | 8279.81 | 16.96 | 14.35 |
| 10.35.191.89 | 185.125.190.122 | 36 | 23 | 9572 | 6106 | 265.89 | 265.48 | 8270.90 | 15.67 | 10.01 |
| 10.35.191.89 | 185.125.190.121 | 1 | 0 | 230 | 0 | 230.00 | 0.00 | N/A | N/A | N/A |
| **Total** | - | 201 | 179 | 48134 | 425

| Client IP | NTP Server IP | # Queries | # Responses | Query Size (bytes) | Response Size (bytes) | Avg Query Size | Avg Response Size | Time Span (s) | Queries/hr | Responses/hr |
|-----------|----------------|-----------|--------------|--------------------|--------------------|----------------|-----------------|---------------|------------|--------------|
| 10.35.191.89 | 91.189.91.113 | 45 | 43 | 10486 | 10026 | 233.02 | 233.16 | 7623.27 | 21.25 | 20.31 |
| 10.35.191.89 | 91.189.91.112 | 40 | 40 | 9200 | 9200 | 230.00 | 230.00 | 7523.33 | 19.14 | 19.14 |
| 10.35.191.89 | 91.189.91.111 | 40 | 40 | 9200 | 9200 | 230.00 | 230.00 | 8407.89 | 17.13 | 17.13 |
| 10.35.191.89 | 185.125.190.123 | 39 | 33 | 9446 | 7998 | 242.21 | 242.36 | 8279.81 | 16.96 | 14.35 |
| 10.35.191.89 | 185.125.190.122 | 36 | 23 | 9572 | 6106 | 265.89 | 265.48 | 8270.90 | 15.67 | 10.01 |
| 10.35.191.89 | 185.125.190.121 | 1 | 0 | 230 | 0 | 230.00 | 0.00 | N/A | N/A | N/A |
| **Total** | - | 201 | 179 | 48134 | 42530 | 239.47 | 237.60 | 40105.21 | 90.21 | 80.34 |

In [6]:
#thsi analyzes only the TLS handshake packages

x=analyze_nts_tcp_pcap('final-nts-ntp.pcap')
print_ntp_summary_markdown(x)

| Client IP | NTP Server IP | # Queries | # Responses | Query Size (bytes) | Response Size (bytes) | Avg Query Size | Avg Response Size | Time Span (s) | Queries/hr | Responses/hr |
|-----------|----------------|-----------|--------------|--------------------|--------------------|----------------|-----------------|---------------|------------|--------------|
| 10.35.191.89 | 91.189.91.111 | 11 | 10 | 1231 | 2096 | 111.91 | 209.60 | 0.45 | 87951.14 | 79955.58 |
| 10.35.191.89 | 185.125.190.121 | 10 | 10 | 1165 | 2096 | 116.50 | 209.60 | 0.10 | 347560.80 | 347560.80 |
| **Total** | - | 21 | 20 | 2396 | 4192 | 114.10 | 209.60 | 0.55 | 273008.46 | 260008.05 |


In [9]:
x=90.21*239
y=90*9.37
print(x/y)

25.566453219494843


| Client IP | NTP Server IP | # Queries | # Responses | Query Size (bytes) | Response Size (bytes) | Avg Query Size | Avg Response Size | Time Span (s) | Queries/hr | Responses/hr |
|-----------|----------------|-----------|--------------|--------------------|--------------------|----------------|-----------------|---------------|------------|--------------|
| 10.35.191.89 | 91.189.91.111 | 11 | 10 | 1231 | 2096 | 111.91 | 209.60 | 0.45 | 87951.14 | 79955.58 |
| 10.35.191.89 | 185.125.190.121 | 10 | 10 | 1165 | 2096 | 116.50 | 209.60 | 0.10 | 347560.80 | 347560.80 |
| **Total** | - | 21 | 20 | 2396 | 4192 | 114.10 | 209.60 | 0.55 | 273008.46 | 260008.05 |

## Comparison

* `timesyncd` uses only 1 IP address and sends 9.37 queries per hour in this 49925s (1.4h) experiment
* `chrony` uses *6* IP addresses and sends between 12 and 21 queries per hour per server in 8400s (2.3h)  -- in total 90 per hour, in this 2.3h experiment
   * the difference is a traffic growth of 9.6x
   * Plus 10 TLS packets per server 
* In volume, the difference will be 25x, given NTS packets are larger
   * `timesyncd:` : `9.37pph*90 bytes`
   * `chrony`: `90.21pph*239 bytes`
   * ratio= 25.56
   